In [99]:
# import keras.callbacks
from music21 import *

import re
import xml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dense, Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
with open('environment.txt') as f:
    env = f.readlines()
# you may also want to remove whitespace characters like \n at the end of each line
env = [x.strip() for x in env]

us = environment.UserSettings()
us['musicxmlPath'] = env[0]
us['musescoreDirectPNGPath'] = env[0]



In [3]:
# fix random seed for reproducibility
np.random.seed(7)

In [4]:
bachRegex = re.compile(r'.*bwv.*', re.IGNORECASE)
bd = corpus.search(bachRegex)

In [112]:
def convertSongToDataFrame(song):
    df = pd.DataFrame()
    
    currentKeySignature = None
    currentTimeSignature = None
    for measure in song.measures(0, None, collect=('TimeSignature','KeySignature')):
        if measure.keySignature:
            currentKeySignature = measure.keySignature
        if measure.timeSignature:
            currentTimeSignature = measure.timeSignature.ratioString
        for i in measure.iter:
            if type(i) is chord.Chord:
                df2 = pd.DataFrame([currentKeySignature, currentTimeSignature, roman.romanNumeralFromChord(i, currentKeySignature).figure, i.duration.quarterLength, i.offset + measure.offset])
                df = df.append(df2[0], ignore_index = True)
    return df


,Unnamed: 0,0,1,2,3,4
0,0,C major,4/4,ii,0.50,0.00
1,1,C major,4/4,ii7,0.50,0.50
2,2,C major,4/4,iii532,0.50,1.00
3,3,C major,4/4,iii,0.25,1.50
4,4,C major,4/4,iii42,0.25,1.75
5,5,C major,4/4,vi6,0.50,2.00
6,6,C major,4/4,#vo6,0.50,2.50
7,7,C major,4/4,vi,0.50,3.00
8,8,C major,4/4,IV6,0.25,3.50
9,9,C major,4/4,vi,0.25,3.75


In [31]:
only run if you want to create csv files
import os.path
import os
if not os.path.exists('./data'):
    os.makedirs("./data")
for s in bd:
    fname = "./data/" + s.sourcePath.split("/")[1]+'.csv'
    if os.path.isfile(fname):
        continue
    df = convertSongToDataFrame(s.parse().chordify())
    df.to_csv(fname, sep=',')
    
#failed at bwv276.mxl

AttributeError: 'KeySignature' object has no attribute 'tonic'

In [75]:

scores = []
import glob, os
os.chdir("C:/Users/errpo/Desktop/projects/project folder/music_prediction/data")
for file in glob.glob("*.csv"):
    scores.append(file)
scoreDf = []

for i in scores:
    score = pd.read_csv(i,usecols=[1,2,3,4,5])
    scoreDf.append(score)
    
# print(scoreDf[1])
scores = pd.concat(scoreDf)

keySignatures = pd.DataFrame([scores['0'].astype('category').cat.codes.unique()], columns=scores['0'].unique())
timeSignatures = pd.DataFrame([scores['1'].astype('category').cat.codes.unique()], columns=scores['1'].unique())
chords = pd.DataFrame([scores['2'].astype('category').cat.codes.unique()], columns=scores['2'].unique())

scores['0'] = scores['0'].astype('category').cat.codes
scores['1'] = scores['1'].astype('category').cat.codes
scores['2'] = scores['2'].astype('category').cat.codes
print(scores)

   F major  g minor  d minor  b minor  A major  D major  a minor  G major  \
0        7       15       12       10        0        4        9        8   

   C major  B- major  E major  e minor  E- major  f# minor  A- major  c minor  
0        3         2        5       13         6        14         1       11  
     0  1    2     3       4
0    7  4   99  1.00    0.00
1    7  4  329  0.50    1.00
2    7  4  329  0.50    1.50
3    7  4   99  0.50    2.00
4    7  4   99  0.50    2.50
5    7  4  774  0.50    3.00
6    7  4  774  0.50    3.50
7    7  4  122  0.50    4.00
8    7  4  122  0.50    4.50
9    7  4  237  0.25    5.00
10   7  4  237  0.25    5.25
11   7  4  291  0.25    5.50
12   7  4  599  0.25    5.75
13   7  4  262  0.50    6.00
14   7  4  851  0.50    6.50
15   7  4   99  1.00    7.00
16   7  4   99  1.00    8.00
17   7  4  237  0.50    9.00
18   7  4  583  0.25    9.50
19   7  4  273  0.25    9.75
20   7  4  895  0.50   10.00
21   7  4  321  0.25   10.50
22   7  4  886  0.

In [37]:
# pd.read_csv('bwv1.6.mxl.csv')

In [76]:
#LSTM
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [84]:
data = scores.values
examples = 100
y_examples = 10
nb_samples = len(data) - examples - y_examples


In [85]:
input_list = [np.expand_dims(np.atleast_2d(data[i:examples+i,:]), axis=0) for i in range(nb_samples)]
input_mat = np.concatenate(input_list, axis=0)


In [86]:
target_list = [np.atleast_2d(data[i+examples:examples+i+y_examples,0]) for i in range(nb_samples)]
target_mat = np.concatenate(target_list, axis=0)


In [100]:
trials = input_mat.shape[0]
features = input_mat.shape[2]
hidden = 64
model = Sequential()
model.add(LSTM(hidden, input_shape=(examples, features)))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(y_examples))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='rmsprop')

ValueError: Input 0 is incompatible with layer flatten_1: expected min_ndim=3, found ndim=2

In [88]:
history = LossHistory()
model.fit(input_mat, target_mat, nb_epoch=100, batch_size=400, callbacks=[history])

c:\users\errpo\anaconda3\envs\music\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
17575/17575 [==============================] - 16s 899us/step - loss: 51.3070
Epoch 2/100
17575/17575 [==============================] - 14s 778us/step - loss: 29.0591
Epoch 3/100
17575/17575 [==============================] - 13s 734us/step - loss: 16.7254
Epoch 4/100
17575/17575 [==============================] - 13s 734us/step - loss: 12.0184
Epoch 5/100
17575/17575 [==============================] - 13s 758us/step - loss: 8.4377
Epoch 6/100
17575/17575 [==============================] - 14s 782us/step - loss: 6.0516
Epoch 7/100
17575/17575 [==============================] - 14s 788us/step - loss: 4.6727
Epoch 8/100
17575/17575 [==============================] - 13s 734us/step - loss: 4.0363
Epoch 9/100
17575/17575 [==============================] - 13s 764us/step - loss: 3.5882
Epoch 10/100
17575/17575 [==============================] - 13s 738us/step - loss: 3.2321
Epoch 11/100
17575/17575 [==============================] - 13s 764us/step - loss: 3.0629
Epoch 12/100
17

17575/17575 [==============================] - 13s 730us/step - loss: 1.8300
Epoch 93/100
17575/17575 [==============================] - 13s 727us/step - loss: 1.7831
Epoch 94/100
17575/17575 [==============================] - 13s 733us/step - loss: 1.7834
Epoch 95/100
17575/17575 [==============================] - 13s 729us/step - loss: 1.7987
Epoch 96/100
17575/17575 [==============================] - 13s 735us/step - loss: 1.8051
Epoch 97/100
17575/17575 [==============================] - 13s 730us/step - loss: 1.8043
Epoch 98/100
17575/17575 [==============================] - 13s 727us/step - loss: 1.7780
Epoch 99/100
17575/17575 [==============================] - 13s 732us/step - loss: 1.7818
Epoch 100/100
17575/17575 [==============================] - 13s 734us/step - loss: 1.8042


In [97]:
examples

100